In [5]:
from PIL import Image
import numpy as np

def convert_image_to_bin(image_path, output_bin_path, target_size=(112, 112)):
	try:
		# 1. Mở và resize ảnh
		img = Image.open(image_path).convert("RGB")
		img = img.resize(target_size)

		# 2. Chuẩn hóa: từ [0, 255] → [-1, 1]
		img_array = np.array(img).astype(np.float32)
		img_array = img_array / 127.5 - 1.0

		# 3. Ghi thành file nhị phân .bin
		img_array.flatten().tofile(output_bin_path)

		print(f"[✓] Đã lưu vào: {output_bin_path}. Shape: {img_array.shape}, dtype: float32")

	except Exception as e:
		print(f"[✗] Lỗi khi xử lý ảnh: {e}")


In [6]:
image_path = "input.jpg"
output_bin_path = "input.bin"

convert_image_to_bin(image_path, output_bin_path)

[✓] Đã lưu vào: input.bin. Shape: (112, 112, 3), dtype: float32


In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Đường dẫn file
IMAGE_PATH = "label.jpg"
OUTPUT_TXT_PATH = "label.txt"
MODEL_PATH = "embedding_model.tflite"  # Đổi thành .h5 nếu bạn dùng Keras

# Kích thước ảnh đầu vào của model (tùy thuộc vào model bạn dùng)
IMG_SIZE = (96, 96)  # Ví dụ với MobileFaceNet

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()

# Lấy chi tiết input/output
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load và tiền xử lý ảnh
img = Image.open(IMAGE_PATH).convert('RGB')
img = img.resize(IMG_SIZE)
img_array = np.array(img, dtype=np.float32) / 255.0  # chuẩn hóa [0, 1]
img_array = np.expand_dims(img_array, axis=0)  # thêm batch dimension

# Gán input cho model
interpreter.set_tensor(input_details[0]['index'], img_array)

# Chạy mô hình
interpreter.invoke()

# Lấy embedding
embedding = interpreter.get_tensor(output_details[0]['index'])[0]  # shape: (192,)

# Lưu vào label.txt
with open(OUTPUT_TXT_PATH, 'w') as f:
	for val in embedding:
		f.write(f"{val:.6f} ")

print(f"Embedding vector saved to {OUTPUT_TXT_PATH}")


ValueError: Could not open 'embedding_model.tflite'.